# Скрипт сбора готовой разметки со всех студенческих папок

## Назначение

**Упрощает повторяющуюся рутинную работу по ручному сбору, скачиванию и распаковке и объединению содержимого архивов разметки, сделанной всеми студентами.**

Собирает все сделанные разметки в один архив All_annotations.zip

## Использование

Перед запуском на своем Google-диске в "Доступных мне" найдите папку "Dataset" и перетащите ее в корень своего Google-диска, т.е. в папку "Мой диск". Затем смонтируйте диск скриптом ниже:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Проверка перед запуском

После монтирования откройте файловую панель Collab слева и убедитесь, что вы видите путь "/drive/MyDrive/Dataset" со всеми подпапками и файлами внутри.

Теперь можно запускать основной скрипт.

Итоговый файл-архив All_annotations.zip сохранится в папку "/Dataset" и будет доступен всем студентам.

In [ ]:
import os
import zipfile

LABEL_FOLDERS = ['Разметка', 'разметка']
BASE_DATASET_PATH = '/content/drive/MyDrive/Path_to_Dataset_Part1'
AIRPORT_DATASET_PATH = '/content/drive/MyDrive/MyDrive/Path_to_Dataset_Part2'

ALL_ANNOTATIONS_ZIP = '/content/drive/MyDrive/Dataset/All_annotations.zip'

passed_annotations = []     # Список успешно добавленных архивов
total_labels = 0            # Всего собрано меток

# Создаем или перезаписываем итоговый архив
with zipfile.ZipFile(ALL_ANNOTATIONS_ZIP, "w") as new_zip:

  # Обходим базовый датасет
  user_folders = os.listdir(BASE_DATASET_PATH)
  sorted_user_folders = sorted(user_folders, key=lambda x: x.lower())  # Сортировка папок без учета регистра

  # Обходим все папки студентов в датасете
  for user_folder in sorted_user_folders:

    # Проверяем наличие подпапки с разметкой в разных варинтах написания
    for label_folder in LABEL_FOLDERS:
      label_path = f'{BASE_DATASET_PATH}/{user_folder}/{label_folder}'

      # В папке студента нашли подпапку 'Разметка' (или похожую)
      if os.path.exists(label_path):

        # Получаем все zip-архивы, которые лежат в подпапке 'Разметка'
        for root, dirs, files in os.walk(label_path):
          for filename in files:
            if filename.endswith(".zip"):

              archive_path = os.path.join(root, filename)
              with zipfile.ZipFile(archive_path, "r") as source_archive:

                # Получаем список txt-файлов в zip-архиве из подпапки /obj_train_data
                obj_train_data_files = [
                  file for file in source_archive.namelist()
                  if file.startswith("obj_train_data/") and file.endswith(".txt")
                ]

                # Добавляем файлы из /obj_train_data в итоговый архив
                for obj_file in obj_train_data_files:
                  file_data = source_archive.read(obj_file)
                  new_path = os.path.join('base_dataset/', os.path.basename(obj_file))
                  new_zip.writestr(new_path, file_data)

              found_labels = len(obj_train_data_files)
              total_labels += found_labels

              # Если нашли в архиве подпапку /obj_train_data и файлы меток в ней
              if found_labels > 0:
                # то добавляем имя обработанного архива с метками в соотв. список
                passed_annotations.append(f'{user_folder}/{label_folder}/{filename} - {found_labels} меток')


  # Обходим датасет из аэропорта
  user_files = os.listdir(AIRPORT_DATASET_PATH)
  sorted_user_files = sorted(user_files, key=lambda x: x.lower())  # Сортировка папок без учета регистра

  # Обходим все папки студентов в датасете
  for filename in sorted_user_files:
    if filename.endswith(".zip"):
      archive_path = f'{AIRPORT_DATASET_PATH}/{filename}'

      with zipfile.ZipFile(archive_path, "r") as source_archive:

        # Получаем список txt-файлов в zip-архиве из подпапки /obj_train_data
        obj_train_data_files = [
          file for file in source_archive.namelist()
          if file.startswith("obj_train_data/") and file.endswith(".txt")
        ]

        # Добавляем файлы из /obj_train_data в итоговый архив
        for obj_file in obj_train_data_files:
          file_data = source_archive.read(obj_file)
          new_path = os.path.join('airport_dataset/', os.path.basename(obj_file))
          new_zip.writestr(new_path, file_data)

      found_labels = len(obj_train_data_files)
      total_labels += found_labels

      # Если нашли в архиве подпапку /obj_train_data и файлы меток в ней
      if found_labels > 0:
        # то добавляем имя обработанного архива с метками в соотв. список
        passed_annotations.append(f'{filename} - {found_labels} меток')

  # Добавляем в итоговый zip-архив текстовый файл со списком обработанных меток
  content = '\n'.join(passed_annotations)
  new_zip.writestr('Список обработанных меток.txt', content.encode())

print(content)
print('\nВсего собрано меток:', total_labels)